# Time To Bet

## Introduction


C'est la 70ème minute dans le classique PSG-OM et vous en êtes sûr : le PSG est certes mené 1-0, mais au vu de sa possession écrasante et de sa quantité pharaonique d'occasions, le score n'en restera pas là. Plus encore, vous allez jusqu'à mettre votre main à couper qu'il y aura 3-1. 
A la 89ème minute, l'histoire vous a donné raison : il ne reste plus que quelques minutes et le PSG mène 3-1. Vous vous mordez les doigts : _vous auriez dû parier_. 

La prochaine fois, n'attendez pas ! Consultez TimeToBet !

On vous dira quel score final semble le plus probable, et s'il est intéressant de miser dessus au vu de sa cote.

Si vous souhaitez vous lancer dans les paris, vous devez savoir comment une cote est calculée. Dans l'idéal, c'est l'inverse de la probabilité. Bien évidemment, les sites de paris diminuent sa vraie valeur afin de s'assurer un gain final. Mais cette relation n'est pas si loin de la réalité et c'est pourquoi pour décider s'il faut parier ou non, nous chercherons à estimer la probabilité d'un score final : si elle est supérieure à l'inverse de la cote (et qu'on aime le risque), alors il faut mettre la main au porte-monnaie.

## Prérequis


In [ ]:
## s'il vous manque un module, installez-le ici
!pip install fuzzywuzzy

In [9]:
import monmodule

## Récupérer les données

Il y a deux moments pour la récupération des données : la phase de collecte de données pour l'entraînement du modèle et la phase de collecte des seules données nécessaires à la visualisation.


#### La collecte des données pour l'entraînement du modèle

A l'aide de sélénium, nous nous connectons sur le site matchendirect.fr qui donne les statistiques d'une grande quantité   de matchs en direct dans le monde. La démarche est la suivante :
- **get_stats()** lance au maximum une fois par minute la fonction **select_champ()** qui boucle sur les championnats.
- **select_champ()** ouvre les url de tous les championnats sélectionnés et récupère la liste de tous les matchs en direct (ils ont un marqueur sépcifique dans le code html : *class='sl '*). Si la liste est vide, on passe au championnat suivant, sinon on lance **select_game()**
- **select_game()** récupère les url de tous les matchs en direct d'un championnat et lance **infos_game()** à chaque fois
- **infos_game()** ouvre l'url d'un match donné et récupère son tableau de statistiques pour l'exporter vers un fichier csv appelé *"Equipe1_Equipe2_Minute"* pour le rendre unique.

Habituellement, cette collecte se fait sur les 5 championnats suivants : Ligue 1, Ligue 2, Liga, Serie A et Bundesliga. La totalité des matchs de ces championnats possèdent des statistiques précises. Toutefois, il est possible de rajouter un autre championnat à étudier en inscrivant son lien en argument de **get_stats()**, ou d'enlever un des championnats principaux de la recherche.  

In [ ]:
## pour boucler sur les 5 championnats principaux
monmodule.get_stats()

In [ ]:
## pour boucler sur les 5 championnats et un autre championnat
monmodule.get_stats({'Serie B': 'https://www.matchendirect.fr/italie/serie-b/'})

In [ ]:
## pour boucler sur seulement 4 des 5 championnats
monmodule.get_stats(champs=['L1', 'L2', 'Liga', 'SerieA'])

#### La collecte des données pour la visualisation


D’abord un process_url assez classique qui ouvre n’importe quel url grâce à la fonction Request comme vu en cours. Cette fonction sert pour les URL Betclic mais aussi Matchendirect et sera donc utilisé tout au long de notre script.
Ensuite, notre fonction fetch_bet_urls prend en intrant l’URL d’un championnat sur Betclic et renvoie toutes les URL des matchs en direct de ce championnat sous forme de liste. Pour cela, on se sert de la classe « app-live-event » dans laquelle on trouve les liens des matchs actuellement en direct. Nous avions un problème avec cette fonction car, si on a un seul match en direct pour un championnat, cette fonction nous renvoie une liste contenant une seule string et, quand on fera ensuite une boucle for sur cette liste, le code prendra en compte que la liste contient les URLs « h », « t », « t », « p », « s », etc. plutôt qu’un seul URL. En ce sens, une boucle conditionnelle sur la longueur de notre liste permettant de rajouter une trailing comma, et ainsi préciser au programme que notre objet est seule dans sa liste.
Le principe de **get_game_odd**, **get_game_teams**, **get_game_name** et **get_game_time** est similaire : on cherche des attributs précis sur notre page. Cela étant, notons que get_game_odd a une particularité : les côtes proviennent d’un JS et non pas d’un simple HTML facilement obtenable via des fonctions recherche. Grâce à l’aide de Salomé Do, nous avons pu régler le souci que nous avions en évitant de lire le fichier comme un script et en le considérant plutôt comme un string.
Note rajoutée le 15/12 : Betclic a fait une grosse maintenance le 14/12 et a changé la manière dont est affiché le temps puisque maintenant le « liveScoreboard_dateTime » est à aller chercher à l’intérieur d’un script lui aussi et cela complexifie la tâche. La fonction **get_game_time** est désormais obsolète et renvoie toujours un « NA ». Ainsi, quand nous aurons besoin du temps dans la suite, nous utiliserons plutôt celui de Matchendirect.
La fonction **get_odds** a pour but de créer un DataFrame avec toutes les données d’un match à la minute où elle est appelée, tandis que la fonction **create_odds_file** récupère les côtes d'un match chaque minute pendant n minutes, avec n = threshold, puis, une fois qu'il a fini, crée un fichier csv par match avec une ligne correspondant aux côtes pour une minute du match. file_nb correspond au nombre de fois où l'on veut répéter cette opération (qui dure threshold minutes), ainsi cette fonction doit durer threshold * file_nb minutes et crée un nombre file_nb de fichiers. Notons que le fichier file_nb = k + 1 contient toutes les lignes du fichier file_nb = k.
La fonction **main** permet de télécharger les fichiers dont j’ai besoin (c’est-à-dire ceux de toutes les ligues choisies) toutes les 10 minutes. Les fichiers créés pour un même match sont certes redondants (exemple : les fichiers de la 21e contiennent ceux de la 11e), mais je préférais cela pour créer des « sauvegardes » et limiter les pertes en cas d’arrêt du code (que parfois je laissais tourner seul, sans être chez moi).

#### Relier les équipes sur Betclic et sur Matchendirect


Dans le cadre de notre plateforme de Data visualisation, nous avions besoin de faire un pont entre le nom des équipes sur Betclic et celui sur Matchendirect. Or, nous ne disposions pas d’ID d’équipe. Ainsi, pour cela, nous avons utilisé le module fuzzywuzzy dont une la fonction process.extractOne permet de savoir quel est le string le plus proche d’un string rentré en input, parmi une liste de strings. Comme les noms entre les deux sites varient assez peu, le programme était très efficace, sauf pour l’équipe de « Mayence » (nom Betclic) qui s’appelle « Mainz 05 » sur Matchendirect.

## Retraiter les données

Un seul fichier, *'alldata'*, centralise les informations utiles contenues dans tous les fichiers de type *'Equipe1_Equipe2_Minute'*. La démarche pour constituer ce fichier est la suivante :

- **gather_data()** récupère la liste de tous les fichiers csv présents dans le répertoire et lance **final_scores()**, puis **find_scoreless()**, puis **complete_data()** avant de créer le fichier csv *'alldata'* et de l'acutaliser sur github avec **to_git()**
- **final_scores()** renvoie, à partir de la liste des fichiers csv, la liste de ceux qui donnent les statistiques les plus avancées (dans le temps) pour chaque match.
- **find_scoreless()** récupère cette liste pour filtrer les matchs dont les informations ne seront pas exploitables (par exemple, si on ne possède des informations que jusqu'à la 85ème minute, on considèrera que le score final aura de grandes chances d'avoir évolué et on ne prendra pas ce match en compte).
- à ce stade, les données de tous les matchs dont l'issue est fiable sont compilées dans un même DataFrame
- **complete_data()** rajoute des lignes manquantes. En effet, bien souvent, il y a tellement de matchs à étudier en même temps qu'il est impossible de récupérer toutes leurs statistiques en moins d'une minute. Dans ce cas-là, il peut arriver qu'il nous manque les données d'un match pour une minute. On pourrait alors s'en passer mais on perdrait de l'information précieuse pour notre modèle alors qu'on peut très facilement et fiablement la calculer à partir des statistiques obtenues à la minute précédente et la minute suivante. La fonction récupère donc pour chaque match la liste des minutes pour lesquelles les statistiques font défaut et ajoute au DataFrame la ligne manquant à partir de valeurs recalculées. Pour finir, complete_data ajoute la variable 'Time' au DataFrame (en récupérant la minute des statistiques), cela permettra par la suite au modèle de comprendre qu'il devient de moins en probable de marquer n buts plus le match avance. 
- **to_git()** est décrite à la fin du rapport, elle permet ici si on le souhaite d'actualiser automatiquement la version 'alldata' de github.

L'argument *update* de **gather_data()** permet de choisir si l'on préfère recréer le fichier *'alldata'* à partir de 0 (utile si l'on a modifié manuellement les données de certains matchs déjà présents dans *'alldata'*) ou simplement ajouter les données des derniers matchs étudiés.

(Attention, en raison de la grande quantité de données, cette étape est très chronophage).

In [ ]:
# Pour exécuter ces fonctions, il est nécessaire d'avoir plusieurs fichiers 
# comme ceux dans le dossier 'data' du repository github, et que ceux-ci
# soient stockées dans le m^mee directory que ce notebook

In [7]:
## à partir de zéro
monmodule.gather_data('alldata', update=False, to_git_or_not_to_git=False)

In [5]:
## à partir de zéro
monmodule.gather_data('alldata', update=True, to_git_or_not_to_git=False)

De nombreux exemples de fichiers sont à disposition du lecteur dans le dossier *'data'* du dépôt. Il était impossible d'en mettre plus de 1000, mais l'ensemble des données sont disponibles dans le fichier *'alldata'*.

## Modélisation


L'objectif du projet étant de prédire le score d'un match à partir de ses statistiques, il nous faut établir un modèle de prédiction. Nous avons choisi la régression logistique multiclasses de ***sklearn*** : pour chaque match, les statistiques de chaque équipe (à une minute précise) seront étudiées séparément afin de prévoir le nombre de buts qu'elle aura marqués à la fin du match. Plus précisément:

- à partir d'un DataFrame contenant les statistiques d'un ou plusieurs matchs, **predict()** lance **prepare_data()**, **regs()** puis **likely_scores()** et renvoie une prédiction de score ainsi que sa probabilité estimée.
- **prepare_data()** divise le DataFrame en deux avec **sep_by_team()** (un DataFrame par équipe, ie par ligne) puis divise chacun de ces deux DataFrames avec **sep_by_time()** en autant de DataFrames qu'il y a de minutes différentes deands. A la fin, nous nous retrouvons donc avec une liste de DataFrames contenant chacun les données statistiques d'une minute pour toutes les équipes à domicile ou toutes les équipes à l'extérieur.
- **regs()** crée un modèle de régression logistique multiclasses pour chaque DataFrame. Chaque DataFrame est standardisé avant que l'on fasse en sorte d'adapter le modèle au DataFrame. Dans le même temps, on récupère 'alldata' dont on ne conserve que les lignes adaptées (même minute que le DataFrame, même type d'équipe) avant de le diviser en un train_set pour entraîner le modèle et un test_set pour tester le modèle. 
- une fois les modèles entraînés sur les données agrégées, on leur demande de prédire les scores de match dont on ne connaît pas l'issue. 
- on récupère les scores potentiels et les probabilités associés à chacun en filtrant les scores impossibles (0-0 score final alors qu'il y a déjà 1-1 à la 70ème par exemple) avec **likely_scores()** et on renvoie le score dont la probabilité est maximale. On peut ici ajouter une fonction qui permet d'ajuster la prédiction du modèle en rendant les scores difficiles à atteindre (écart entre le score final et le score actuel) moins probables.

En réalité, on ne récupère pas *une* prédiction et *une* probabilité (celles du score final) mais *deux* : la prédiction et la probabilité du nombre de buts inscrits *par chaque équipe*. Nous avons choisi d'utiliser l'hypothèse simplificatrice que la probabilité du score final était égale au produit des probabilités des nombres de buts de chaque équipe (nous n'avons pas pris en compte la dépendance).

In [7]:
## télécharger le fichier souhaité du dossier data
filename = input('nom du fichier')
monmodule.predict(monmodule.pd.read_csv(filename), 'alldata')

nom du fichier"Atalanta_Fiorentina_69'"


FileNotFoundError: [Errno 2] File b'"Atalanta_Fiorentina_69\'"' does not exist: b'"Atalanta_Fiorentina_69\'"'

## Visualisation

On a choisi d’utiliser le module *Dash*, qui permettent de développer une plateforme Web sans devoir coder de fichier HTML, CSS (ou JS) à côté car tout passe par Python. Ayant déjà manié ce module interactif en stage, je trouvais cela intéressant d’avoir un livrable sous forme de plateforme dans laquelle l’utilisateur sélectionne le match qui l’intéresse et nous lui affichons les informations, ainsi que nos prédictions et leur rapport aux côtes pour lui indiquer quand parier. Pour développer la plateforme, nous avons d’abord réalisé une maquette, qui a certes évolué avec le temps, mais qui nous fournissait une base claire de l’objectif final.

Dash repose sur un fonctionnement qu’il semble indispensable de comprendre : le **callback**. Ces fonctions prennent en input les paramètres de la page dont elles doivent observer le moindre changement (par l’utilisateur ou, pour l’élément Interval, par le temps écoulé), et sortent en output de nouvelles valeurs pour des éléments définis par l’utilisateur. Une fois qu’on a défini un callback, le programme applique la fonction qui suit dans le code (qui est la plus proche voisine sous le callback) et l’applique en prenant pour valeurs de la fonction les états des inputs.
Le callback le plus important et le plus difficile à saisir à première lecture est celui qui se fonde sur la fonction **get_stat_df**. Nous l’avons changé en urgence le 15/12 après la mise à jour de Betclic car celui-ci ne pouvait plus aller scraper le temps du match comme avant. Nous avons donc réuni différents callbacks en un seul pour ne pas avoir à faire le même travail plusieurs fois. Ce qu’il faut toujours avoir à l’esprit pour mieux comprendre ce callback, c’est que les variables suivies d’un 1 correspondent à « odds-table » (tableau des côtes) tandis que les variables suivies d’un 2 comptent pour « stat-table ».

Pour rafraîchir la page toutes les minutes, nous voulions, en première instance, utiliser un time.sleep(60 – le temps de notre programme). Toutefois, mettre le programme en sommeil posait un problème quant à notre fonction **main** et notre **run_server**. Ainsi, nous avons d’abord réfléchi à avoir recours à du multithreading ou du multiprocessing (après avoir essayé de comprendre les différences entre les deux procédés), puis, Eureka, nous avons trouvé un élément Dash Core Components, appelé Interval, qui a précisément cette utilité : il compte le temps, en millisecondes, qui passent à partir du lancement du programme et, chaque fois qu’il arrive au bout du laps de temps défini par l’argument interval, il augmente de 1 l’argument n_intervals. Je ne me sers jamais de la valeur de ce dernier mais simplement du fait qu’il change, pour indiquer à mes callbacks qu’ils doivent se rafraîchir.

In [10]:
if __name__ == '__main__':
    monmodule.app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "monmodule" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

C:\Users\louis\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Pour aller plus loin


Evidemment, nous n'avons pas abouti à ce résultat en *oneshot* et nous avons donc certains morceaux de code qui ne sont plus nécessaires au produit final mais que nous avons jugé intéressants de mentionner ici.

#### to_git()


Plutôt que d'écrire à chaque fois les mêmes commandes pour actualiser un fichier sur github, nous avons créé la fonction **to_git()** qui inscrit ces commandes elle-même. A l'aide de *pyautogui*, elle prend le contrôle de votre clavier pour taper les commandes à votre place. Renseignez-lui les *noms des fichiers* que vous voulez modifier, leur *emplacement*, votre *clef ssh* et son *mot de passe* (stocké dans un fichier en local), et elle fera ce travail fastidieux toute seule. 
Attention : elle ne gère pas les problèmes de conflit et n'est utile que lorsqu'on est le seul à avoir retravaillé un fichier.

In [ ]:
## voir le fichier monmodule

#### red_card()

Une erreur s'était glisée dans le code lors de la collecte de la première moitié des données : nous n'avons pas pu collecter d'informations sur les remplacements et les cartons rouges. Rerécupérer les informations sur les remplacements semblait impossible et finalement peu utile (on se doute bien que c'est une information bonus pour prédire le score d'un match), nous avons donc choisi de retirer cette variable de nos données, le temps de se constituer une base de données assez conqéuente pour pouvoir supprimer les données desmatchs pour lesquels nous n'avons pas cette information. En revanche, le nombre de cartons rouges apparaît bien plus déterminant et nous avons donc créé une fonction qui ouvre l'url de chaque match déjà étudié et renvoie ceux dans lesquels un carton rouge a été distribué. Il n'y en avait qu'un. Et il avait été donné en toute fin de match. Nous avons donc rajouté manuellement cette information sur les quelques fichiers (énumérables sur les doigts d'une main) nécessaires. A présent, toutes les statistiques sont bien collectées et cette fonction ne semble plus avoir d'usage immédiat.

In [11]:
monmodule.red_card()

set()


#### Pour une suite au projet

Le projet se base aujourd'hui sur environ 80 matchs. Cette base de données a vocation à s'agrandir au fil des weekends, ce qui permettra de rendre le modèle beaucoup plus performant. Par exemple, à l'heure actuelle, la base de données ne contient qu'un match dans lequel une équipe a marqué 5 buts, et elle est parvenue à le faire en tirant au but ... ***4*** fois. Malheureusement pour le moment la classe 5 buts n'est déterminée que par ce match, et lorsque le modèle prédit 5 buts, cela ne nous ressemble par conséquent que très peu significatif. 

Nous avons également songé à des pistes d'amélioration :
- créer un dossier par match afin de pouvoir les stocker plus facilement sur github
- mettre un historique des matchs précédents d'une équipe sélectionnée sur la plateforme
- faire tourner la plateforme en ligne et en continu pour qu'un utilisateur qui se connecte puisse avoir toutes les données du match (depuis son coup d'envoi) et voir l'évolution des statistiques
- ajouter des informations 'insolites' quand la masse de données sera plus conséquente : par exemple, nos données nous permettraient de savoir à quelle minute d'un match il est le plus probable qu'une équipe à domicile prenne un carton rouge !

## Conclusion

Nous sommes pleinement satisfaits par notre choix de projet et par le produit final. Certes, certaines prédictions semblent aujourd'hui improbables, mais nous avons bon espoir de penser qu'en continuant à collecter des données, elle seront de plus en plus précises. Par ailleurs, rappelons que l'objectif premier du projet n'était pas de trouver la meilleure formule ou le meilleur modèle pour prédire un score final afin de réussir tous ses paris : nous savons bien qu'à ce petit jeu, il est impossible de battre les bookmakers. Au contraire, nous avons trouvé un grand intérêt à nous focalis.er sur des données différentes de celles principalement regardées par les sites de paris (à savoir le classement des équipes et l'historique des derniers matchs de chaque équipe, ou encore le rapport entre le nombre de buts marqués sur le nombre de buts pris au cours de la période étudiée). D'ailleurs, nous pensions d'abord récupérer les données statistiques des matchs minute par minute avant de nous rendre compte que de telles données étaient introuvables (ou alors très bien cachées). 